# CSCI 3220: Python for Data Science and AI

## Assignment 3

## Due Date: April 11(Monday), 11:59 PM

## Total Points: 35

## Bonus Points: 5

#### ***Write the code in Jupyter notebook (ipynb file) with proper comments. 
#### ***Add proper citation if you take help from a different source (not from the textbook).
#### ***Rename the file with your student ID and submit it in Moodle.

In [4]:
import pandas as pd
import numpy as np
import sklearn
import math
import collections

## 1a.	Load any one of the toy datasets (for classification) from scikit-learn library.

In [5]:
from sklearn.datasets import load_iris

iris = load_iris()

## 1b. Display general information and statistics about your selected dataset.

In [6]:
print(iris.DESCR)

.. _iris_dataset:

Iris plants dataset
--------------------

**Data Set Characteristics:**

    :Number of Instances: 150 (50 in each of three classes)
    :Number of Attributes: 4 numeric, predictive attributes and the class
    :Attribute Information:
        - sepal length in cm
        - sepal width in cm
        - petal length in cm
        - petal width in cm
        - class:
                - Iris-Setosa
                - Iris-Versicolour
                - Iris-Virginica
                
    :Summary Statistics:

    ============== ==== ==== ======= ===== ====================
                    Min  Max   Mean    SD   Class Correlation
    ============== ==== ==== ======= ===== ====================
    sepal length:   4.3  7.9   5.84   0.83    0.7826
    sepal width:    2.0  4.4   3.05   0.43   -0.4194
    petal length:   1.0  6.9   3.76   1.76    0.9490  (high!)
    petal width:    0.1  2.5   1.20   0.76    0.9565  (high!)
    ============== ==== ==== ======= ===== ===========

## 2.	Split the dataset into train and test sets.

In [7]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(iris.data, iris.target)

## 3.	Implement a k-nearest neighbors classifier (k =3) for the training set. Show the result for the test set using the following metric: accuracy, precision, and recall.

In [8]:
#Gets Distance Between Two Observations
def getDistance(point1, point2):    
    distance_sepal_length = ((point1[0] - point2[0]) ** 2)
    distance_sepal_width = ((point1[1] - point2[1]) ** 2)
    distance_petal_length = ((point1[2] - point2[2]) ** 2)
    distance_petal_width = ((point1[3] - point2[3]) ** 2)
    distance = math.sqrt(distance_sepal_length + distance_sepal_width + distance_petal_length + distance_petal_width) 
    return distance

#Determines Species by KNN
def determineSpecies(point, x_train, y_train, n):
    distanceTrainObservationPair = dict()
    
    #Add Distance to Dictionary Observations
    for train_observation in range(len(x_train)):
        distanceTrainObservationPair.update({getDistance(point, x_train[train_observation]) : train_observation})
        
    #Citation: Learned How to Sort Dictionary on StackOverflow
    #Link: https://stackoverflow.com/questions/9001509/how-can-i-sort-a-dictionary-by-key
    distanceTrainObservationPair = collections.OrderedDict(sorted(distanceTrainObservationPair.items()))
    distanceTrainObservationPair = dict(distanceTrainObservationPair)
    
    #Increments Index of Species Array to Lowest Three Observation's Label
    species = [0, 0, 0]
    for i in range(n):
        train_observation_index = list(distanceTrainObservationPair.values())[i]
        species[y_train[train_observation_index]] = species[y_train[train_observation_index]] + 1
     
    #Determine Which Species is Nearest Neighbors
    current_largest_n_species = -1 
    for i in range(len(species)):  
        if(species[i] > current_largest_n_species):
            current_largest_n_species = i
    
    return current_largest_n_species


#Guess Species for Each Observation in x_test and Save to y_result
y_predictions = np.arange(len(x_test)) #Determined Results by KNN
for i in range(len(x_test)):
    y_predictions[i] = determineSpecies(x_test[i], x_train, y_train, 3)

print("Predicted Results:\n", y_predictions)
print("True Results:\n", y_test)

Predicted Results:
 [0 0 1 0 0 2 0 1 0 2 2 2 0 1 2 2 2 0 2 2 1 1 0 1 0 2 0 0 2 0 1 0 2 0 1 2 1
 1]
True Results:
 [0 0 1 0 0 1 0 1 0 2 2 2 0 1 2 2 2 0 2 2 1 1 0 1 0 2 0 0 2 0 2 0 2 0 1 2 1
 1]


In [9]:
#Calculate Accuracy
def getAccuracy(y_predictions, y_test):
    correct_number_of_predictions = 0
    for i in range(len(y_predictions)):
        if(y_predictions[i] == y_test[i]):
            correct_number_of_predictions = correct_number_of_predictions + 1
    return (correct_number_of_predictions/len(y_predictions))*100

def getConfusionMatrix(y_predictions, y_test):
    num_true_positives = 0
    num_true_negatives = 0
    num_false_positives = 0
    num_false_negatives = 0
    
    for i in range(len(y_predictions)):
        pass

print("Accuracy: %.2f%%" %(getAccuracy(y_predictions, y_test)))

Accuracy: 94.74%


In [10]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_predictions)

array([[15,  0,  0],
       [ 0,  9,  1],
       [ 0,  1, 12]])

## 4.	Split the training set into 10-fold for cross-validation. Calculate accuracy for each held-out set. Also, display the average accuracy for ten folds.

In [33]:
#print(y_train)

x_fold = list()
y_fold = list()

#Create Folds of Xs and Ys of Length 10
for i in range(0, len(x_train), 10):
    x_subset = x_train[i:i+10]
    x_fold.append(x_subset)
    y_subset = y_train[i:i+10]
    y_fold.append(y_subset)

#Train and Test using KFold
for i in range(len(y_fold)-1):
    #Guess Species for Each Observation in x_test and Save to y_result
    y_predictions = np.arange(len(x_test)) #Determined Results by KNN
    test_x = x_fold[i]
    test_y = y_fold[i]
    
    for j in range(len(x_test)):
        if j == i:
            test_x = x_fold[j]
            test_y = y_fold[j]
        else:
            y_predictions[j] = determineSpecies(x_test[i], x_train, y_train, 3)

## 5.	Find the optimal value of k ranging from 1 to 10 based on mean accuracy of 10-fold cross-validation.

## 6. Implement the steps (3-5) with the scikit-learn library method and compare it with your previous results.

In [20]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_iris

iris6 = load_iris()

x_train6, x_test6, y_train6, y_test6 = train_test_split(iris6.data, iris6.target)

KNearestNeighbors = KNeighborsClassifier(n_neighbors = 1)
KNearestNeighbors.fit(x_train6, y_train6)

y_predictions6 = KNearestNeighbors.predict(x_test6)

print("Predicted Results:\n", y_predictions6)
print("True Results:\n", y_test6)

Predicted Results:
 [1 1 1 0 1 0 1 2 1 2 2 2 2 0 1 2 1 0 1 0 2 0 1 2 0 2 1 0 2 2 0 1 2 2 1 0 0
 1]
True Results:
 [1 1 1 0 1 0 1 2 1 2 2 2 2 0 1 2 1 0 1 0 2 0 1 2 0 2 1 0 2 2 0 1 2 2 1 0 0
 1]


In [22]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.model_selection import KFold

print("Accuracy: %.2f%%" %(accuracy_score(y_test6, y_predictions6) * 100))
print("Precision: %d" %(precision_score(y_test6, y_predictions6, average="macro")))
print("Recall: %d" %(recall_score(y_test6, y_predictions6, average="macro")))

Accuracy: 100.00%
Precision: 1
Recall: 1
